<a href="https://colab.research.google.com/github/AlexKazmin/SkillFactory-Projects/blob/main/GD3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GD-3. Исследование поведения пользователей, зарегестрированных в 2018 году

1. Получаем данные из файлов

In [ ]:
import pandas as pd
import numpy as np

purchase = pd.read_csv('data/GD-3/purchase.csv', sep=',')
events = pd.read_csv('data/GD-3/7_4_Events.csv', sep=',')


2. Производим преобразование и фильтрацию

In [ ]:
# создать фильтр 2018
cond2018 = (events.start_time>='2018-01-01') & (events.start_time<'2019-01-01') & (events.event_type=='registration')
registered = events[cond2018]  
# список пользователей, зарег. в 2018
lst_reg = registered['user_id'].to_list() 
# отфильтровать пользователей зарег в 2018
events_df = events[events.user_id.isin(lst_reg)].copy() 
purchase_df = purchase[purchase.user_id.isin(lst_reg)].copy() 
# добавить ивент тип покупка
purchase_df['event_type'] = 'purchase' 
# изменить тип 
purchase_df['event_datetime'] = pd.to_datetime(purchase_df['event_datetime'])
# переименовать айди
events_df = events_df.rename(columns={"id": "event_id"}) 
purchase_df = purchase_df.rename(columns={"id": "purchase_id"})
# объединить датафреймы
total_events_df = pd.concat([events_df,purchase_df],sort=False) 
# изменить тип 
total_events_df['start_time'] = pd.to_datetime(total_events_df['start_time'])
total_events_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68559 entries, 51405 to 2778
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   event_id        66959 non-null  float64       
 1   event_type      68559 non-null  object        
 2   selected_level  8342 non-null   object        
 3   start_time      66959 non-null  datetime64[ns]
 4   tutorial_id     32954 non-null  float64       
 5   user_id         68559 non-null  int64         
 6   purchase_id     1600 non-null   float64       
 7   event_datetime  1600 non-null   datetime64[ns]
 8   amount          1600 non-null   float64       
dtypes: datetime64[ns](2), float64(4), int64(1), object(2)
memory usage: 5.2+ MB


3. Выделяем группы пользователей

In [ ]:
user_levels = total_events_df[~total_events_df['selected_level'].isnull()].copy()
user_levels_df = user_levels[['selected_level', 'user_id', 'start_time']]
group_easy = user_levels_df[(user_levels_df['selected_level'] == 'easy')]["user_id"].unique()
group_medium = user_levels_df[(user_levels_df['selected_level'] == 'medium')]["user_id"].unique()
group_hard = user_levels_df[(user_levels_df['selected_level'] == 'hard')]["user_id"].unique()
set_easy = set(group_easy) #список выбравших низкий уровень сложности
set_medium = set(group_medium) #список выбравших средний уровень сложности
set_hard = set(group_hard) #список выбравших высокий уровень сложности

4. Рассчитываем для каждой группы процент оплат

In [ ]:
# сколько пользователей в каждой группе по уровню сложности
groups_df = user_levels_df['selected_level'].value_counts().to_frame().reset_index()
groups_df = groups_df.rename(columns={"selected_level": "users_count"})
groups_df = groups_df.rename(columns={"index": "selected_level"})
# добавить покупки к тем, кто выбрал уровень
merged_df_1 = user_levels_df.merge(purchase_df, on='user_id', how='outer')
# расчет количества тех, кто совершил покупку по группам уровней
bought_groups_df = merged_df_1.groupby('selected_level')['purchase_id'].count().to_frame().reset_index()
bought_groups_df = bought_groups_df.rename(columns={"purchase_id": "purchase_count"})
# процент оплат для каждой группы
level_stats_df = groups_df.merge(bought_groups_df, on='selected_level', how='outer')
level_stats_df['purchase_percent'] = (level_stats_df['purchase_count'] / level_stats_df['users_count']*100).round(decimals=2)
# вывод получившейся таблицы
print(level_stats_df)

  selected_level  users_count  purchase_count  purchase_percent
0         medium         4645             969             20.86
1           easy         2448             189              7.72
2           hard         1249             442             35.39


### Вывод: чем выше уровень трудности выбрал пользователь, тем выше вероятность, что он совершит покупку.

5. Рассчитываем временной промежуток между событиями регистрации и оплаты

In [ ]:
# переименовать колонку event_datetime на start_time в таблице purchase_df чтобы подготовить к объединению 
purchase_df_2 = purchase_df.copy().rename(columns={"event_datetime": "start_time"})
total_events_2 = pd.concat([events_df,purchase_df_2],sort=False) # объединить датафреймы

# здесь можем суммировать даты потому что для всех событый кроме прохождения туториала есть только ОДНА дата
a = total_events_2.pivot_table(
    values='start_time',
    index= ['user_id'],
    columns= 'event_type',
    aggfunc='sum')

# вытаскиваем user_id из индекса в колонку
a.reset_index(level=0, inplace=True)
# убираем лишние колонки
del a["tutorial_finish"]
del a["tutorial_start"]
del a["pack_choice"] 
# убираем все строки где нет покупок
a = a[~a['purchase'].isnull()].copy()
# меняем тип на дату
a['registration'] = pd.to_datetime(a['registration'])
a['level_choice'] = pd.to_datetime(a['level_choice'])

# расчет времени между событиями для пользователей выбравших низкий уровень сложности
easy_df = a[a["user_id"].isin(set_easy)].copy()
easy_df['timedelta'] = easy_df['purchase'] - easy_df['registration'] 
easy_df['timedelta2'] = easy_df['level_choice'] - easy_df['registration'] 
easy_time = easy_df['timedelta'].mean()

# расчет времени между событиями для пользователей выбравших средний уровень сложности
medium_df = a[a["user_id"].isin(set_medium)].copy()
medium_df['timedelta'] = medium_df['purchase'] - medium_df['registration'] 
medium_df['timedelta2'] = medium_df['level_choice'] - medium_df['registration'] 
medium_time = medium_df['timedelta'].mean()

# расчет времени между событиями для пользователей выбравших высокий уровень сложности
hard_df = a[a["user_id"].isin(set_hard)].copy()
hard_df['timedelta'] = hard_df['purchase'] - hard_df['registration'] 
hard_df['timedelta2'] = hard_df['level_choice'] - hard_df['registration'] 
hard_time = hard_df['timedelta'].mean()

# вывод данных
print(f'''Временной промежуток между регистрацией и оплатой у групп пользователей с:
1) низким уровнем сложности: {easy_time},
2) средним уровнем сложности: {medium_time},
3) высоким уровнем сложности: {hard_time}''')

Временной промежуток между регистрацией и оплатой у групп пользователей с:
1) низким уровнем сложности: 3 days 22:10:23.211640211,
2) средним уровнем сложности: 4 days 06:12:06.576883384,
3) высоким уровнем сложности: 3 days 14:55:19.257918552


### Выоды: Нет существенной разницы во временных промежутках между регистрацией и оплатой, в зависимости от уровня сложности. Все временные интервалы находятся в диапазоне +-3 дня.
### Быстрее всех совершают покупку пользователи, выбравшие высокий уровень сложности (за 3 дня и 15 часов).
